In [1]:
# import autosklearn.regression
import sklearn.model_selection
import sklearn.metrics
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

<h2>02 Data Collection</h2>

In [3]:
df = []
src = r"C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\train\train"
for file in os.listdir(src):
    df.append(pd.read_csv(os.path.join(src,file)))
for i in df:
    print(list(i))
df = pd.concat(df)
df = df.fillna(0)
df[-10:]

['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '29SoilTension', 'flow1', '26N', '26P', '26K']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
104,12_105,04:40:16.090733+00,44.1,31.2,0.0,47.64,0.0,0.0,0.5,3.0,38.8,22.8,0.11,5.0,21.0,95.0,87.0,105.0,0.0
105,12_106,04:42:16.132055+00,44.2,31.2,0.0,49.50,0.0,0.0,0.1,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,111.0,0.0
106,12_107,04:44:16.107327+00,45.2,30.8,0.0,50.03,0.0,0.0,1.2,7.0,37.4,22.8,0.11,5.0,21.0,95.0,87.0,113.0,0.0
107,12_108,04:46:16.11276+00,45.9,32.0,0.0,50.06,0.0,0.0,0.6,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,110.0,0.0
108,12_109,04:48:16.120498+00,41.4,33.1,0.0,50.36,0.0,0.0,0.7,7.0,37.6,22.8,0.11,5.0,21.0,95.0,87.0,108.0,0.0
109,12_110,04:50:16.123489+00,43.0,33.0,0.0,52.30,0.0,0.0,1.0,5.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,109.0,0.0
110,12_111,04:52:18.185763+00,42.5,32.5,0.0,53.76,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.0,21.0,95.0,87.0,0.0,0.0
111,12_112,04:54:16.141528+00,41.7,33.1,0.0,54.86,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,109.0,0.0
112,12_113,04:56:20.078654+00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,2.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,104.0,0.0
113,12_114,04:58:16.116687+00,44.8,32.5,0.0,58.81,0.0,0.0,1.2,7.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,113.0,0.0


In [4]:
df.shape

(1111, 19)

In [5]:
df.columns

Index(['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh',
       '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed',
       '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน',
       '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1'],
      dtype='object')

In [6]:
df.describe(include='all')

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
count,1111,1111,1111.000000,1111.000000,1111.000000,1111.000000,1111.0,1111.0,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000,1111.000000
unique,1111,1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,01_1,01:00:50.174233+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,49.581548,28.236184,20.820963,25.309307,0.0,0.0,0.734383,3.021602,42.186679,21.774077,0.118875,6.585329,20.355536,89.074707,82.429343,41.747075,0.411341
std,NaN,NaN,12.457318,4.691715,30.524152,24.089878,0.0,0.0,0.762373,2.263594,12.820314,1.824412,0.020832,1.159089,7.291150,30.236438,28.100947,33.303057,0.492298
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,41.400000,25.000000,0.000000,4.840000,0.0,0.0,0.000000,1.000000,34.300000,21.400000,0.110000,5.000000,21.000000,94.000000,87.000000,23.000000,0.000000
50%,NaN,NaN,47.000000,28.500000,0.000000,10.390000,0.0,0.0,0.600000,2.000000,37.600000,21.900000,0.110000,7.000000,21.000000,96.000000,89.000000,29.000000,0.000000
75%,NaN,NaN,55.300000,31.200000,65.530000,53.200000,0.0,0.0,1.100000,5.000000,45.200000,22.900000,0.120000,7.300000,23.000000,99.000000,92.000000,33.000000,1.000000


<hr>

<h4>03 Exploratory Data Analysis (EDA)</h4>

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1111 entries, 0 to 113
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1111 non-null   object 
 1   time                    1111 non-null   object 
 2   1Air Humidity           1111 non-null   float64
 3   1AirTemperature         1111 non-null   float64
 4   1LightHigh              1111 non-null   float64
 5   1LightLow               1111 non-null   float64
 6   11Rain Accumulate (mm)  1111 non-null   float64
 7   11Rain count            1111 non-null   float64
 8   12WindSpeed             1111 non-null   float64
 9   13WindDirection         1111 non-null   float64
 10  23SoilWC ค.ชื้นดิน      1111 non-null   float64
 11  23SoilTemp.อุณหภูมิดิน  1111 non-null   float64
 12  23SoilEC                1111 non-null   float64
 13  23SoilpH                1111 non-null   float64
 14  26N                     1111 non-null   float6

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
1Air Humidity,1111.0,49.581548,12.457318,0.0,41.40,47.00,55.30,97.30
1AirTemperature,1111.0,28.236184,4.691715,0.0,25.00,28.50,31.20,38.40
1LightHigh,1111.0,20.820963,30.524152,0.0,0.00,0.00,65.53,65.53
1LightLow,1111.0,25.309307,24.089878,0.0,4.84,10.39,53.20,65.52
11Rain Accumulate (mm),1111.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.00
11Rain count,1111.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.00
12WindSpeed,1111.0,0.734383,0.762373,0.0,0.00,0.60,1.10,5.00
13WindDirection,1111.0,3.021602,2.263594,0.0,1.00,2.00,5.00,7.00
23SoilWC ค.ชื้นดิน,1111.0,42.186679,12.820314,0.0,34.30,37.60,45.20,80.20
23SoilTemp.อุณหภูมิดิน,1111.0,21.774077,1.824412,0.0,21.40,21.90,22.90,23.60


In [10]:
# # Pair Plot
# sns.pairplot(df)
# plt.show()

In [11]:
def remove_outliers(df):
    cleaned_data = df.copy()
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            cleaned_data = cleaned_data[(cleaned_data[col] >= lower_bound) & (cleaned_data[col] <= upper_bound)]
    return cleaned_data

In [12]:
# Remove outliers from the dataset
df = remove_outliers(df)
df

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
21,01_22,01:44:50.183143+00,75.1,28.1,0.00,46.37,0.0,0.0,0.0,0.0,51.4,23.5,0.12,8.3,23.0,101.0,94.0,23.0,1.0
49,01_50,02:40:50.233795+00,48.0,32.3,0.00,62.16,0.0,0.0,0.0,2.0,58.5,23.4,0.12,7.8,25.0,104.0,97.0,22.0,0.0
50,01_51,02:42:50.232726+00,45.7,33.0,0.00,61.97,0.0,0.0,0.0,4.0,57.9,23.3,0.12,7.8,25.0,103.0,96.0,21.0,0.0
51,01_52,02:44:50.259068+00,44.8,34.4,0.00,62.74,0.0,0.0,0.0,2.0,53.3,23.4,0.12,7.8,25.0,103.0,96.0,21.0,0.0
52,01_53,02:46:50.262598+00,45.5,34.4,0.00,63.44,0.0,0.0,0.0,3.0,52.1,23.4,0.12,7.8,24.0,103.0,96.0,21.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,08_113,04:39:08.909415+00,35.6,37.9,65.53,2.46,0.0,0.0,0.4,4.0,37.4,21.8,0.11,6.7,21.0,94.0,87.0,23.0,0.0
113,08_114,04:41:08.913736+00,33.8,38.2,65.53,2.99,0.0,0.0,0.2,0.0,36.9,21.8,0.11,6.7,21.0,94.0,87.0,24.0,0.0
114,08_115,04:43:08.879486+00,34.3,38.4,65.53,1.92,0.0,0.0,0.2,2.0,36.9,21.8,0.11,6.6,21.0,94.0,87.0,24.0,0.0
115,08_116,04:45:08.87669+00,37.3,36.5,65.53,1.77,0.0,0.0,0.3,1.0,36.9,21.8,0.11,6.7,21.0,94.0,87.0,23.0,0.0


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
1Air Humidity,568.0,47.864613,9.738520,32.30,39.30,46.700,54.6250,75.40
1AirTemperature,568.0,28.082394,4.691330,17.70,24.50,27.600,31.6000,38.40
1LightHigh,568.0,30.688345,32.727947,0.00,0.00,0.000,65.5300,65.53
1LightLow,568.0,15.892289,21.192172,0.00,2.93,6.655,12.8525,65.52
11Rain Accumulate (mm),568.0,0.000000,0.000000,0.00,0.00,0.000,0.0000,0.00
11Rain count,568.0,0.000000,0.000000,0.00,0.00,0.000,0.0000,0.00
12WindSpeed,568.0,0.671831,0.651831,0.00,0.00,0.600,1.1000,2.70
13WindDirection,568.0,3.098592,2.253932,0.00,1.00,3.000,5.0000,7.00
23SoilWC ค.ชื้นดิน,568.0,38.159859,5.845130,29.80,35.00,37.250,40.6000,61.20
23SoilTemp.อุณหภูมิดิน,568.0,21.410211,1.015194,19.90,20.50,21.600,22.0000,23.50


In [14]:
# # Pair Plot
# sns.pairplot(df)
# plt.show()

<h4>04 Feature Engineering</h4>

In [15]:
df.columns

Index(['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh',
       '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed',
       '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน',
       '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1'],
      dtype='object')

In [16]:
df

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
21,01_22,01:44:50.183143+00,75.1,28.1,0.00,46.37,0.0,0.0,0.0,0.0,51.4,23.5,0.12,8.3,23.0,101.0,94.0,23.0,1.0
49,01_50,02:40:50.233795+00,48.0,32.3,0.00,62.16,0.0,0.0,0.0,2.0,58.5,23.4,0.12,7.8,25.0,104.0,97.0,22.0,0.0
50,01_51,02:42:50.232726+00,45.7,33.0,0.00,61.97,0.0,0.0,0.0,4.0,57.9,23.3,0.12,7.8,25.0,103.0,96.0,21.0,0.0
51,01_52,02:44:50.259068+00,44.8,34.4,0.00,62.74,0.0,0.0,0.0,2.0,53.3,23.4,0.12,7.8,25.0,103.0,96.0,21.0,0.0
52,01_53,02:46:50.262598+00,45.5,34.4,0.00,63.44,0.0,0.0,0.0,3.0,52.1,23.4,0.12,7.8,24.0,103.0,96.0,21.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,08_113,04:39:08.909415+00,35.6,37.9,65.53,2.46,0.0,0.0,0.4,4.0,37.4,21.8,0.11,6.7,21.0,94.0,87.0,23.0,0.0
113,08_114,04:41:08.913736+00,33.8,38.2,65.53,2.99,0.0,0.0,0.2,0.0,36.9,21.8,0.11,6.7,21.0,94.0,87.0,24.0,0.0
114,08_115,04:43:08.879486+00,34.3,38.4,65.53,1.92,0.0,0.0,0.2,2.0,36.9,21.8,0.11,6.6,21.0,94.0,87.0,24.0,0.0
115,08_116,04:45:08.87669+00,37.3,36.5,65.53,1.77,0.0,0.0,0.3,1.0,36.9,21.8,0.11,6.7,21.0,94.0,87.0,23.0,0.0


<h5>Feature Importance</h5>

In [17]:
df.shape

(568, 19)

In [18]:
dfx = df
list(dfx)

['index',
 'time',
 '1Air Humidity',
 '1AirTemperature',
 '1LightHigh',
 '1LightLow',
 '11Rain Accumulate (mm)',
 '11Rain count',
 '12WindSpeed',
 '13WindDirection',
 '23SoilWC ค.ชื้นดิน',
 '23SoilTemp.อุณหภูมิดิน',
 '23SoilEC',
 '23SoilpH',
 '26N',
 '26P',
 '26K',
 '29SoilTension',
 'flow1']

In [19]:
dfx = dfx.drop(['index','time','11Rain Accumulate (mm)','11Rain count'],axis=1)

In [20]:
dfx.corr().style.format("{:.4}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
1Air Humidity,1.0,-0.6864,-0.5912,-0.1602,-0.113,-0.08394,-0.1843,0.1519,-0.07761,-0.01946,-0.12,-0.102,-0.1058,0.3544,0.4695
1AirTemperature,-0.6864,1.0,0.4786,0.0571,-0.2407,0.1268,0.3227,0.4598,0.1113,0.2015,0.381,0.3651,0.3637,-0.5419,-0.4221
1LightHigh,-0.5912,0.4786,1.0,-0.4956,0.2242,-0.02699,0.04584,0.04864,-0.01923,0.2419,0.1436,0.08823,0.1112,-0.5386,-0.4593
1LightLow,-0.1602,0.0571,-0.4956,1.0,0.01673,-0.01426,0.2213,-0.14,0.1664,-0.1909,0.1215,0.1319,0.1226,0.1455,-0.02137
12WindSpeed,-0.113,-0.2407,0.2242,0.01673,1.0,-0.01827,-0.06725,-0.2704,0.01206,-0.03756,-0.103,-0.1124,-0.1051,-0.006826,-0.0101
13WindDirection,-0.08394,0.1268,-0.02699,-0.01426,-0.01827,1.0,0.08436,0.06515,0.09482,-0.0739,0.0005585,0.00222,0.02956,0.0559,0.01407
23SoilWC ค.ชื้นดิน,-0.1843,0.3227,0.04584,0.2213,-0.06725,0.08436,1.0,0.3139,0.7221,0.02623,0.636,0.7294,0.7171,-0.1545,-0.171
23SoilTemp.อุณหภูมิดิน,0.1519,0.4598,0.04864,-0.14,-0.2704,0.06515,0.3139,1.0,0.1584,0.6606,0.6513,0.6471,0.653,-0.5026,-0.02591
23SoilEC,-0.07761,0.1113,-0.01923,0.1664,0.01206,0.09482,0.7221,0.1584,1.0,-0.06099,0.346,0.4227,0.4302,0.1458,0.1491
23SoilpH,-0.01946,0.2015,0.2419,-0.1909,-0.03756,-0.0739,0.02623,0.6606,-0.06099,1.0,0.6031,0.5394,0.5679,-0.6401,-0.1244


<ol>
    <li>26P             0.7294</li>
    <li>23SoilEC        0.7221	</li>
    <li>26K             0.7171</li>
    <li>26N             0.636</li>
    <li>1AirTemperature 0.3227</li>
    <li>1LightLow       0.2213</li>
    <li>13WindDirection	       0.08436</li>
    <li>1LightHigh       0.04584</li>
    <li>23SoilpH	       0.02623</li>
</ol>

In [21]:
# X = dfx[['1Air Humidity', '1AirTemperature', '1LightHigh',
#                 '1LightLow','12WindSpeed',
#                 '23SoilEC', '23SoilpH', '23SoilTemp.อุณหภูมิดิน',  '26N', '26P', '26K',
#                 '29SoilTension', 'flow1']]
# X = dfx[['26P','23SoilpH','26K','26N','1AirTemperature','1LightLow']]
# X = dfx[['26P','23SoilpH','26K','26N','1AirTemperature','1LightLow','1LightHigh','23SoilEC']]
X = dfx[['23SoilEC','26N', '26P', '26K']]
y = df[['23SoilWC ค.ชื้นดิน']]

In [22]:
X

,23SoilEC,26N,26P,26K
21,0.12,23.0,101.0,94.0
49,0.12,25.0,104.0,97.0
50,0.12,25.0,103.0,96.0
51,0.12,25.0,103.0,96.0
52,0.12,24.0,103.0,96.0
...,...,...,...,...
112,0.11,21.0,94.0,87.0
113,0.11,21.0,94.0,87.0
114,0.11,21.0,94.0,87.0
115,0.11,21.0,94.0,87.0


In [23]:
y

,23SoilWC ค.ชื้นดิน
21,51.4
49,58.5
50,57.9
51,53.3
52,52.1
...,...
112,37.4
113,36.9
114,36.9
115,36.9


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 99)
X_train.shape, X_test.shape

((397, 4), (171, 4))

<h4>Scaling?</h4>

In [26]:
from sklearn.preprocessing import RobustScaler

In [27]:
cols = X_train.columns

In [28]:
X_train

,23SoilEC,26N,26P,26K
4,0.11,20.0,93.0,86.0
101,0.11,21.0,95.0,87.0
33,0.12,22.0,99.0,91.0
28,0.11,21.0,95.0,87.0
81,0.11,21.0,94.0,87.0
...,...,...,...,...
112,0.11,21.0,96.0,88.0
13,0.11,21.0,96.0,89.0
79,0.11,22.0,97.0,89.0
99,0.11,21.0,96.0,88.0


In [29]:
# scaler = RobustScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [30]:
# X_train = pd.DataFrame(X_train, columns=[cols])
# X_test = pd.DataFrame(X_test, columns=[cols])

<hr>

<h4>05 Model Selection</h4>

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [32]:
np.linspace(0,1,num=20)

array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])

In [33]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import BayesianRidge
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [35]:
# 01Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)
lr_mse = mean_squared_error(y_test, lr_predictions)


In [36]:
# 02 Polynomial Regression (using a pipeline for polynomial feature transformation)
poly_param_grid = {
    'polynomialfeatures__degree': [2, 3, 4],
    'linearregression__fit_intercept': [True, False]
}
poly_pipeline = Pipeline([
    ('polynomialfeatures', PolynomialFeatures()),
    ('linearregression', LinearRegression())
])
poly_grid_search = GridSearchCV(poly_pipeline, poly_param_grid, cv=5, scoring='neg_mean_absolute_error')
poly_grid_search.fit(X_train, y_train)
poly_best = poly_grid_search.best_estimator_
poly_predictions = poly_best.predict(X_test)
poly_mse = mean_squared_error(y_test, poly_predictions)



In [37]:
#03 Ridge Regression
ridge_param_grid = {
    'alpha': [0.1, 1.0, 10.0]
}
ridge_model = Ridge()
ridge_grid_search = GridSearchCV(ridge_model, ridge_param_grid, cv=5, scoring='neg_mean_absolute_error')
ridge_grid_search.fit(X_train, y_train)
ridge_best = ridge_grid_search.best_estimator_
ridge_predictions = ridge_best.predict(X_test)
ridge_mse = mean_squared_error(y_test, ridge_predictions)


In [38]:
# 04 Lasso Regression
lasso_param_grid = {
    'alpha': [0.1, 0.5, 1.0]
}
lasso_model = Lasso()
lasso_grid_search = GridSearchCV(lasso_model, lasso_param_grid, cv=5, scoring='neg_mean_absolute_error')
lasso_grid_search.fit(X_train, y_train)
lasso_best = lasso_grid_search.best_estimator_
lasso_predictions = lasso_best.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_predictions)


In [39]:
# 05 Elastic Net Regression
enr = ElasticNet()
enr_param_grid = {
    'alpha': [0.1, 0.5, 1.0], 
    'l1_ratio': [0.1, 0.5, 0.9]
    }
enr_grid_search = GridSearchCV(enr, enr_param_grid, cv=5, scoring='neg_mean_absolute_error')
enr_grid_search.fit(X_train, y_train)
enr_best = enr_grid_search.best_estimator_
enr_predictions = enr_best.predict(X_test)
enr_mse = mean_squared_error(y_test, enr_predictions)

In [40]:
# 06 Support Vector Regression (SVR)
svr_param_grid = {
    'C': [1.0, 10.0, 100.0],
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf', 'linear']
}
svr_grid_search = GridSearchCV(SVR(), svr_param_grid, cv=5, scoring='neg_mean_absolute_error')
svr_grid_search.fit(X_train, y_train)
svr_best = svr_grid_search.best_estimator_
svr_predictions = svr_best.predict(X_test)
svr_mse = mean_squared_error(y_test, svr_predictions)


In [41]:
# 07 Decision Tree Regression
dt_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20]
}
dt_grid_search = GridSearchCV(DecisionTreeRegressor(), dt_param_grid, cv=5, scoring='neg_mean_absolute_error')
dt_grid_search.fit(X_train, y_train)
dt_best = dt_grid_search.best_estimator_
dt_predictions = dt_best.predict(X_test)
dt_mse = mean_squared_error(y_test, dt_predictions)


In [42]:
# 08 Random Forest Regressor
rfr = RandomForestRegressor()
rfr_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10]
}
rfr_grid_search = GridSearchCV(rfr, rfr_param_grid, cv=5, scoring='neg_mean_absolute_error')
rfr_grid_search.fit(X_train, y_train)
rfr_best = rfr_grid_search.best_estimator_
rfr_predictions = rfr_best.predict(X_test)
rfr_mse = mean_squared_error(y_test, rfr_predictions)


In [43]:
# 09 Gradient Boosting Regression (XGBoost)
xgb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6, 9]
}
xgb_grid_search = GridSearchCV(xgb.XGBRegressor(), xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
xgb_grid_search.fit(X_train, y_train)
xgb_best = xgb_grid_search.best_estimator_
xgb_predictions = xgb_best.predict(X_test)
xgb_mse = mean_squared_error(y_test, xgb_predictions)

In [44]:
# 10 Gradient Boosting Regression (LightGBM)
lgb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6, 9],
    'num_leaves': [31, 62, 127]
}
lgb_grid_search = GridSearchCV(lgb.LGBMRegressor(), lgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
lgb_grid_search.fit(X_train, y_train)
lgb_best = lgb_grid_search.best_estimator_
lgb_predictions = lgb_best.predict(X_test)
lgb_mse = mean_squared_error(y_test, lgb_predictions)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000046 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 36
[LightGBM] [Info] Number of data points in the train set: 317, number of used features: 4
[LightGBM] [Info] Start training from score 38.226183
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [45]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from pygam import LinearGAM, s, f
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [46]:
# 11 Gaussian Process Regression
kernel = 1.0 * RBF(1.0)  # Example kernel
gpr = GaussianProcessRegressor(kernel=kernel)
gpr.fit(X_train, y_train)
gpr_predictions = gpr.predict(X_test)
gpr_mae = mean_absolute_error(y_test, gpr_predictions)






In [47]:
# # K-Nearest Neighbors Regression (KNN)
# knn_param_grid = {
#     'n_neighbors': [3, 5, 7],
#     'weights': ['uniform', 'distance'],
#     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
# }
# knn_grid_search = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=5, scoring='neg_mean_absolute_error')
# knn_grid_search.fit(X_train, y_train)
# knn_best = knn_grid_search.best_estimator_
# knn_predictions = knn_best.predict(X_test)
# knn_mae = mean_absolute_error(y_test, knn_predictions)

In [48]:
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 10, 15],  # Testing more values for the number of neighbors
    'weights': ['uniform', 'distance'],  # Whether to weight points by the inverse of their distance
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
    'leaf_size': [20, 30, 40, 50],  # Leaf size passed to BallTree or KDTree
    'p': [1, 2],  # Power parameter for the Minkowski metric (1 = Manhattan, 2 = Euclidean)
    'metric': ['minkowski', 'euclidean', 'manhattan']  # The distance metric to use
}

# Setup GridSearchCV
knn_grid_search = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
knn_grid_search.fit(X_train, y_train)

# Fetching the best model
knn_best = knn_grid_search.best_estimator_

# Making predictions with the best model
knn_predictions = knn_best.predict(X_test)
knn_mae = mean_absolute_error(y_test, knn_predictions)

print(f'Best KNN Model MAE: {knn_mae}')
print(f'Best Parameters: {knn_grid_search.best_params_}')


Best KNN Model MAE: 1.8217646612578253
Best Parameters: {'algorithm': 'auto', 'leaf_size': 40, 'metric': 'minkowski', 'n_neighbors': 15, 'p': 2, 'weights': 'distance'}


In [49]:
# 12 Neural Network Regression
nn_param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50,50)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'lbfgs']
}
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
nn_grid_search = GridSearchCV(MLPRegressor(max_iter=1000), nn_param_grid, cv=5, scoring='neg_mean_absolute_error')
nn_grid_search.fit(X_train_scaled, y_train)
nn_best = nn_grid_search.best_estimator_
nn_predictions = nn_best.predict(X_test_scaled)
nn_mae = mean_absolute_error(y_test, nn_predictions)

In [50]:
# nn_param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (100, 50), (50, 50), (50, 30, 20)],
#     'activation': ['relu', 'tanh', 'logistic'],
#     'solver': ['adam', 'lbfgs', 'sgd'],  # Added 'sgd' solver
#     'alpha': [0.0001, 0.001, 0.01],  # L2 penalty (regularization term) parameter
#     'learning_rate_init': [0.001, 0.01, 0.1],  # Initial learning rate
#     'max_iter': [1000, 1500, 2000],  # Increased range for max_iter
#     'beta_1': [0.9, 0.95],  # Only used with 'adam' solver
#     'beta_2': [0.999, 0.995],  # Only used with 'adam' solver
#     'epsilon': [1e-8, 1e-7]  # Only used with 'adam' solver
# }

# # Remember to scale your features as neural networks are sensitive to feature scaling
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Setup GridSearchCV
# nn_grid_search = GridSearchCV(MLPRegressor(max_iter=2000), nn_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
# nn_grid_search.fit(X_train_scaled, y_train)

# # Fetching the best model
# nn_best = nn_grid_search.best_estimator_

# # Making predictions with the best model
# nn_predictions = nn_best.predict(X_test_scaled)
# nn_mae = mean_absolute_error(y_test, nn_predictions)

# print(f'Best Neural Network Model MAE: {nn_mae}')
# print(f'Best Parameters: {nn_grid_search.best_params_}')

In [51]:
# 14 Generalized Additive Models (GAM) - Example with LinearGAM
gam = LinearGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
gam_predictions = gam.predict(X_test)
gam_mae = mean_absolute_error(y_test, gam_predictions)

# Quantile Regression
model = sm.QuantReg(y_train, sm.add_constant(X_train))
res = model.fit(q=0.5)
qr_predictions = res.predict(sm.add_constant(X_test))
qr_mae = mean_absolute_error(y_test, qr_predictions)



In [52]:
# Multiple Linear Regression
mlr = LinearRegression()
mlr_param_grid = {}
mlr_grid_search = GridSearchCV(mlr, mlr_param_grid, cv=5, scoring='neg_mean_absolute_error')
mlr_grid_search.fit(X_train, y_train)
mlr_best = mlr_grid_search.best_estimator_
mlr_predictions = mlr_best.predict(X_test)
mlr_mse = mean_squared_error(y_test, mlr_predictions)

In [53]:
# Gradient Boosting Regression Trees
gbrt = GradientBoostingRegressor()
gbrt_param_grid = {
    'n_estimators': [50, 100, 200], 
    'learning_rate': [0.05, 0.1, 0.2]
    }
gbrt_grid_search = GridSearchCV(gbrt, gbrt_param_grid, cv=5, scoring='neg_mean_absolute_error')
gbrt_grid_search.fit(X_train, y_train)
gbrt_best = gbrt_grid_search.best_estimator_
gbrt_predictions = gbrt_best.predict(X_test)
gbrt_mse = mean_squared_error(y_test, gbrt_predictions)

In [54]:
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error

In [55]:
# Poisson Regression
poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
poisson_predictions = poisson_model.predict(X_test)

In [56]:
# Negative Binomial Regression
nb_model = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial()).fit()
nb_predictions = nb_model.predict(X_test)

c:\Users\Acer\anaconda3\envs\tf\lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [57]:
# Print mean squared errors
print(f'Linear Regression MSE: {lr_mse}')
print(f'Polynomial Regression MSE: {poly_mse}, Best Degree: {poly_grid_search.best_params_["polynomialfeatures__degree"]}')
print(f'Ridge Regression MSE: {ridge_mse}, Best Alpha: {ridge_grid_search.best_params_["alpha"]}')
print(f'Lasso Regression MSE: {lasso_mse}, Best Alpha: {lasso_grid_search.best_params_["alpha"]}')
print("Elastic Net Regression MSE:", enr_mse)
print(f'GAM MAE: {gam_mae}') 
print(f'SVR MSE: {svr_mse}, Best Params: {svr_grid_search.best_params_}')
print(f'Decision Tree MSE: {dt_mse}, Best Params: {dt_grid_search.best_params_}')
print("Random Forest Regressor MSE:", rfr_mse)
print(f'XGBoost MSE: {xgb_mse}, Best Params: {xgb_grid_search.best_params_}')
print(f'LightGBM MSE: {lgb_mse}, Best Params: {lgb_grid_search.best_params_}')
print(f'Gaussian Process Regression MAE: {gpr_mae}')
print(f'KNN Regression MAE: {knn_mae}, Best Params: {knn_grid_search.best_params_}')
print(f'Neural Network Regression MAE: {nn_mae}, Best Params: {nn_grid_search.best_params_}')
print(f'Quantile Regression MAE: {qr_mae}')
print("Multiple Linear Regression MSE:", mlr_mse)
print("Gradient Boosting Regression Trees MSE:", gbrt_mse)
print("Poisson Regression MAE:", mean_absolute_error(y_test, poisson_predictions))
print("Negative Binomial Regression MAE:", mean_absolute_error(y_test, nb_predictions))

Linear Regression MSE: 8.675290314809782
Polynomial Regression MSE: 94.65134650524661, Best Degree: 4
Ridge Regression MSE: 14.114405839026084, Best Alpha: 0.1
Lasso Regression MSE: 14.774573883458471, Best Alpha: 0.1
Elastic Net Regression MSE: 14.913235601487917
GAM MAE: 1.9349551251136636
SVR MSE: 12.897331768741347, Best Params: {'C': 100.0, 'gamma': 'auto', 'kernel': 'rbf'}
Decision Tree MSE: 6.473183203411487, Best Params: {'max_depth': None, 'min_samples_split': 2}
Random Forest Regressor MSE: 6.49716867824858
XGBoost MSE: 6.5635498821856695, Best Params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
LightGBM MSE: 6.880989126559357, Best Params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'num_leaves': 31}
Gaussian Process Regression MAE: 1.9181888730403833
KNN Regression MAE: 1.8217646612578253, Best Params: {'algorithm': 'auto', 'leaf_size': 40, 'metric': 'minkowski', 'n_neighbors': 15, 'p': 2, 'weights': 'distance'}
Neural Network Regression MAE:

<hr>

<h4>06 Model Training</h4>

In [58]:
x_test = []
src2 = r'C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test'
for file in os.listdir(src2):
    print(os.path.join(src2,file))
    x_test.append(pd.read_csv(os.path.join(src2,file)))
x_test = pd.concat(x_test)
x_test = x_test.fillna(0)
x_test

C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test\set02.csv
C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test\set09.csv


,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
0,02_1,04:11:15.736838+00,46.2,30.7,65.53,8.79,0.0,0.0,1.3,2.0,22.4,0.11,7.9,21.0,95.0,88.0,26.0,0.0
1,02_2,04:13:15.740386+00,46.0,31.4,65.53,8.26,0.0,0.0,0.0,1.0,22.4,0.11,7.9,21.0,95.0,88.0,26.0,0.0
2,02_3,04:15:17.720768+00,44.4,32.2,65.53,8.66,0.0,0.0,1.0,2.0,22.3,0.11,7.9,21.0,95.0,88.0,0.0,0.0
3,02_4,04:17:15.750584+00,43.0,33.0,65.53,8.68,0.0,0.0,0.0,2.0,22.4,0.11,7.9,21.0,95.0,88.0,26.0,0.0
4,02_5,04:19:15.75123+00,42.4,32.9,65.53,8.86,0.0,0.0,0.0,2.0,22.4,0.11,7.9,21.0,95.0,88.0,26.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,09_112,07:20:44.482378+00,36.2,35.9,0.00,53.35,0.0,0.0,0.0,3.0,23.7,0.12,4.9,27.0,110.0,103.0,48.0,0.0
112,09_113,07:22:46.459288+00,39.8,35.9,0.00,54.13,0.0,0.0,1.9,0.0,23.7,0.12,4.9,27.0,110.0,103.0,0.0,0.0
113,09_114,07:24:44.482942+00,38.4,35.8,0.00,54.11,0.0,0.0,0.0,1.0,23.7,0.12,4.9,27.0,110.0,103.0,48.0,0.0
114,09_115,07:26:44.533862+00,38.2,35.6,0.00,54.26,0.0,0.0,0.0,1.0,23.7,0.12,4.9,27.0,110.0,103.0,45.0,0.0


In [59]:
X.columns

Index(['23SoilEC', '26N', '26P', '26K'], dtype='object')

In [60]:
x_test = x_test[X.columns]
x_test

,23SoilEC,26N,26P,26K
0,0.11,21.0,95.0,88.0
1,0.11,21.0,95.0,88.0
2,0.11,21.0,95.0,88.0
3,0.11,21.0,95.0,88.0
4,0.11,21.0,95.0,88.0
...,...,...,...,...
111,0.12,27.0,110.0,103.0
112,0.12,27.0,110.0,103.0
113,0.12,27.0,110.0,103.0
114,0.12,27.0,110.0,103.0


In [61]:
# scaler = RobustScaler()
# x_test = scaler.fit_transform(x_test)
# x_test = scaler.transform(x_test)
# x_test = pd.DataFrame(x_test, columns=[cols])

In [62]:
y = nn_best.predict(X_test)
y

array([1004.58877433,  976.08572779,  982.82123113,  996.22292773,
        986.15432776, 1029.68941417, 1051.45695737,  996.22292773,
       1014.65737431, 1007.92187096, 1071.59186606,  986.15432776,
        986.15203651, 1029.68941417,  996.22063648,  964.38678456,
        976.08572779,  961.05368793,  996.22292773, 1004.58877433,
       1014.65737431,  961.05368793,  964.38678456, 1024.72597428,
       1029.68941417, 1007.92187096,  976.08572779, 1004.58877433,
        971.1222879 ,  982.82123113,  976.08572779,  996.22292773,
        996.22292773,  971.1222879 , 1007.92187096,  996.22292773,
       1017.99047094,  964.38678456,  986.15432776, 1073.22220932,
        971.1222879 ,  964.38678456, 1007.92187096,  996.22292773,
       1029.68941417,  971.1222879 , 1029.68941417, 1007.92187096,
       1017.99047094, 1014.65737431, 1024.72368303,  976.08572779,
       1014.65737431,  996.22292773, 1014.65737431,  996.22292773,
        964.38678456,  961.05368793,  996.22292773,  954.31818

<hr>

<h4>07 Create SSV</h4>

In [63]:
submission = pd.read_csv(r'C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\sample_submission.csv')
submission

,index,23SoilWC ค.ชื้นดิน
0,02_1,31.1
1,02_2,NaN
2,02_3,NaN
3,02_4,NaN
4,02_5,NaN
...,...,...
224,09_112,NaN
225,09_113,NaN
226,09_114,NaN
227,09_115,NaN


In [64]:
submission['23SoilWC ค.ชื้นดิน'].iloc[1:] = y[1:]
submission

ValueError: cannot set using a slice indexer with a different length than the value

In [ ]:
# submission.to_csv('v.0.0.5.csv',index=False)